In [1]:
import os

In [2]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\credit\\FraudApp\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\credit\\FraudApp'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Godwin45/FraudApp.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Godwin45"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1eaeb42d599213d6a9ab7754a9e2515f5d042e6c"
os.environ['DAGSHUB_TOKEN'] = "f0a0ac483c588c3c561ae0bdf41b55f9bc2dcd48"

# MLFLOW_TRACKING_URI=https://dagshub.com/Godwin45/FraudApp.mlflow \
# MLFLOW_TRACKING_USERNAME=Godwin45 \
# MLFLOW_TRACKING_PASSWORD=1eaeb42d599213d6a9ab7754a9e2515f5d042e6c \
# python script.py


# MLFLOW_TRACKING_URI=https://dagshub.com/Godwin45/FraudApp.mlflow \
# MLFLOW_TRACKING_USERNAME=Godwin45 \
# MLFLOW_TRACKING_PASSWORD=8895347721dd5cd1b0369aba5665f82a77ca7d72 \
# python script.py

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Godwin45/FraudApp.mlflow",
           
        )

        return model_evaluation_config


In [9]:
import numpy as np
import pandas as pd
import mlflow
import joblib
from pathlib import Path
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, precision_score, recall_score, f1_score, accuracy_score
from urllib.parse import urlparse

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        
        precision = precision_score(actual, pred)  # Precision
        recall = recall_score(actual, pred)        # Recall
        f1 = f1_score(actual, pred)                # F1-score
        accuracy = accuracy_score(actual, pred)    # Accuracy
        
        return rmse, mae, r2, precision, recall, f1, accuracy

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1).values
        test_y = test_data[[self.config.target_column]].values.ravel()

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2, precision, recall, f1, accuracy) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {
                "rmse": rmse, "mae": mae, "r2": r2,
                "precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy
            }
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1", f1)
            mlflow.log_metric("accuracy", accuracy)

            # Model registry does not work with the file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="FraudAppModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()

except Exception as e:
    raise e

[2023-08-13 01:21:52,062: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-13 01:21:52,098: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-13 01:21:52,135: INFO: common: yaml file: schema.yaml loaded successfully]


[2023-08-13 01:21:52,143: INFO: common: created directory at: artifacts]
[2023-08-13 01:21:52,154: INFO: common: created directory at: artifacts/model_evaluation]
[2023-08-13 01:22:02,986: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\pcx\AppData\Local\anaconda3\envs\credit\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'FraudAppModel' already exists. Creating a new version of this model...
2023/08/13 01:22:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: FraudAppModel, version 5
Created version '5' of model 'FraudAppModel'.
